In [34]:
import random
import numpy as np

In [35]:
dataset=[10,11]
learningRate=0.0002
errorThresh=1

class Layer:
    
    def __init__(self,neuronCount=None,position=None,neuronvals=None):
        
        self.neuronCount = neuronCount
        self.position = position
        self.neuronvals = []
        self.weights = []
        self.bias = []
        self.NodeDeltas=[] 

        #set by setintialweights and setnodedelta
        self.previousLayer =None
        self.AfterLayer =None

        for i in range(self.neuronCount):

            # self.bias.append(random.uniform(0.0001,1))
            self.bias.append(1)

        if position==1:
            self.neuronvals = neuronvals
    
    def setInitialWeights(self,previousLayer):

        self.previousLayer = previousLayer
        test =[
            [1,2,3,4]
        ]
        for i in range(self.neuronCount):
            temp = []
            for j in range(previousLayer.neuronCount):
                temp.append(random.uniform(0.0001,1))
                # temp.append(test[0][i+j])

            self.weights.append(temp)

    def setNodeDelta(self,AfterLayer):
        """""
        set nodedelta value for layers
        """""
        self.AfterLayer = AfterLayer
        if self.position == -1:
            # outputlayer
            if len(self.NodeDeltas)==0:
                for i in range(self.neuronCount):

                    nodedelta = self.neuronvals[i]-dataset[i] #exepectd to change dimenston CHNGDIMN
                    self.NodeDeltas.append(nodedelta)
            else:
                for i in range(self.neuronCount):

                    nodedelta = self.neuronvals[i]-dataset[i] #exepectd to change dimenston CHNGDIMN
                    self.NodeDeltas[i]=nodedelta
        else:
            # hidden layers
            if len(self.NodeDeltas)==0:

                for i in range(self.neuronCount):
                    nodedelta= 0
                    for j in range(AfterLayer.neuronCount):
                        nodedelta = nodedelta+AfterLayer.NodeDeltas[j]*self.weights[i][j]
                    self.NodeDeltas.append(nodedelta)

            else:
                    for i in range(self.neuronCount):
                        nodedelta= 0
                        for j in range(AfterLayer.neuronCount):
                            nodedelta = nodedelta+AfterLayer.NodeDeltas[j]*self.weights[i][j]
                        self.NodeDeltas[i]=nodedelta
    
    def updateWeightsandBias(self):

        len_weight = len(self.weights[0])# based on previous layer of no(neurons) lenof weights is set

        for i in range(self.neuronCount):

            for j in range(len_weight):
                
                new_Weight = self.weights[i][j] - learningRate * self.NodeDeltas[i] * self.previousLayer.neuronvals[j]
                self.weights[i][j] = new_Weight
            
            new_Bias = self.bias[i] - learningRate * self.NodeDeltas[i]
            self.bias[i]=new_Bias        

In [36]:
class Ann:

    def __init__(self):
        self.LayerArr = []
        self.batchSize = -1

    def strap(self,layer1=None,layer2=None):
        "sets the initial weights of the ANN"
        layer2.setInitialWeights(layer1)

    def forward(self,layer1=None,layer2=None):
        "performs forward passing for all layers in network"  
        if len(layer2.neuronvals)==0:
            for i in range(layer2.neuronCount):
                layer2.neuronvals.append(np.dot(layer1.neuronvals,layer2.weights[i])+layer2.bias[i])
        else:
            for i in range(layer2.neuronCount):
                layer2.neuronvals[i]=np.dot(layer1.neuronvals,layer2.weights[i]+layer2.bias[i])
            
    def backPropagate(self,currentLayer=None,AfterLayer=None):
        """""
        performs back propagation by

        1. seting nodeDelta (a function in lasyer class)
        2.Update werights and biases (a function in layer class)
        
        """""
        
        
        
        currentLayer.setNodeDelta(AfterLayer)
        currentLayer.updateWeightsandBias()
        
    def compile(self,batchSize=-1):
        """""
        calls strap function for every layer : initilizes weights in Ann
        """""
        self.batchSize = batchSize
        for i in range(len(self.LayerArr)-1):
            self.strap(layer1=self.LayerArr[i],layer2=self.LayerArr[i+1])
    
    def findError(self):
        error = 0
        for i in range(len(self.LayerArr[-1].neuronvals)):#CHNGDIMN
            error = error+(dataset[i]-self.LayerArr[-1].neuronvals[i])**2
        error = 0.5*error
        return error
    
    def Train(self):
        """
        Trains the model 
        
        """
        errorrate = 9999999 
        layercount = len(self.LayerArr)

        while (errorrate > errorThresh):

            #forward propagation
            for i in range(layercount-1):
                self.forward(layer1=self.LayerArr[i],layer2=self.LayerArr[i+1])
            
            #error analysis
            errorrate = self.findError()
            # print(errorrate)
            if(errorrate <errorThresh):
                print("optimised")
                print(self.LayerArr[-1].neuronvals)
                break
          

            #backward propagation
            currentLayeridx = layercount-1  
            for i in range(layercount-1):

                if self.LayerArr[currentLayeridx].position ==-1:# handling output layer 
                    self.backPropagate(currentLayer=self.LayerArr[currentLayeridx])
                    print(self.LayerArr[currentLayeridx].neuronvals)
                    currentLayeridx=currentLayeridx-1
                else:
                    self.backPropagate(currentLayer=self.LayerArr[currentLayeridx],AfterLayer=self.LayerArr[currentLayeridx+1])
                    currentLayeridx=currentLayeridx-1
                              
    def bind(self,layer=None):
        """
        use this function to add layers into network
        
        if input layer give coresponding value too
        """
        self.LayerArr.append(layer)

In [37]:

# inputl = Layer(2,1,[2,3])
inputl = Layer(neuronCount=2,position=1,neuronvals=[2,3])
hidden = Layer(neuronCount=2,position=2,neuronvals=[])
# hidden2 = Layer(neuronCount=2,position=3,neuronvals=[]) 
output = Layer(neuronCount=2,position=-1,neuronvals=[])

Ann = Ann()
Ann.bind(layer=inputl)
Ann.bind(layer=hidden)
# Ann.bind(layer=hidden2)
Ann.bind(layer=output)
Ann.compile()
Ann.Train()

[3.473913529766821, 3.59941952519292]
[21.771525863990814, 22.075143800910183]
[21.27110202830697, 21.59551374178667]
[20.799563248709493, 21.143809534912094]
[20.35460265852216, 20.71779523622763]
[19.934155045583836, 20.315469258068013]
[19.53636567876663, 19.935034131229354]
[19.159563869474106, 19.57487086047519]
[18.80224044622298, 19.233517076084674]
[18.463028480029188, 18.90964833891595]
[18.140686723801693, 18.602062078205535]
[17.83408532823399, 18.309663737655303]
[17.54219347572566, 18.031454782046318]
[17.26406863715853, 17.766522278025036]
[16.998847207312274, 17.51402981214837]
[16.745736315952225, 17.27320954929172]
[16.504006645174716, 17.04335526707555]
[16.272986111019456, 16.82381622857154]
[16.052054289875137, 16.61399177739374]
[15.840637488772593, 16.413326557293285]
[15.638204374035336, 16.221306273291496]
[15.444262085540334, 16.037453923786533]
[15.258352774509342, 15.861326443417655]
[15.080050511686869, 15.69251170513896]
[14.908958520271124, 15.530625837240